<a href="https://colab.research.google.com/github/sedwardsmarsh/genetic-algorithm-hello-world/blob/main/genetic_algorithm_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I'm using this notebook to learn about genetic algorithms

## sources:
* [quick and dirty introduction](https://towardsdatascience.com/introduction-to-genetic-algorithms-including-example-code-e396e98d8bf3)
* [wiki page](https://en.wikipedia.org/wiki/Genetic_algorithm)
* [bonkers applications of gen algs](http://alglobus.net/NASAwork/papers/Space2006Antenna.pdf)

## TODO: change probabilities of zero and one generation for population initialization 

In [ ]:
# imports
import numpy as np
from operator import attrgetter

np.random.seed(0)

In [ ]:
class GeneticAlgorithm:

  # internal chromosome class
  class Chromosome:
    def __init__(self, g=None, genes=[]):
      # g is the number of genes per chromosome
      if g == None:
        g = 10
      if genes != []:
        self.genes = genes
      else:
        self.genes = np.random.randint(0, 2, g)
      self.fitness = None

    def __str__(self):
      return str(self.genes)
        
  
  def __init__(self, n=10, g=10):
    self.parents = []
    # n is the population size
    self.population = []
    for i in range(n):
      C = self.Chromosome(g)
      self.population.append(C)

  def update_fitness(self, style='sum'):
      if style == 'sum':
        for i in range(len(self.population)):
          self.population[i].fitness = np.sum(self.population[i].genes)
  
  def sort_population(self):
    self.population.sort(key=lambda c: c.fitness)
  
  def select(self, style='greatest'):
    # if parents % 2 != 0:
    #   print('parents must be multiple of 2')
    self.sort_population()
    for i in range(2):
      # higher probability to select most-fit chromosomes
      if style == 'greatest': 
        if np.random.randint(0, 10) < 8:
          self.parents.append(self.population[-1])
        else: # select random chromosome in population
          self.parents.append(
              self.population[np.random.randint(0, len(self.population))])
          
  def crossover(self, radix=None):
    # define radix as crossover point inside chromosome
    radix = np.random.randint(2, len(self.population)-2)
    # swap parent's genes and create children
    parent1, parent2 = self.parents[0], self.parents[1]
    child1 = self.Chromosome(genes=np.concatenate((parent1.genes[:radix], parent2.genes[radix:])))
    child2 = self.Chromosome(genes=np.concatenate((parent2.genes[:radix], parent1.genes[radix:])))
    # add children to front of population
    self.population.append(child1)
    self.population.append(child2)

  def mutation(self):
    # low probability of mutation
    if np.random.randint(0, 10) > 1:
      return
    child1, child2 = self.population[-1], self.population[-2]
    i = np.random.randint(2, 10) # select random number of genes to mutate
    for c in (child1, child2):
      for x in range(i):
        if i >= len(c.genes): # restrict i to be within length of chromosome
          i -= len(c.genes)
        c.genes[i] = 1 if c.genes[i] == 0 else 0
        i += 1

  def trim(self, style='greatest'):
    if style == 'greatest':
      # remove least fit chromosomes
      del self.population[:2]

  def fitness_difference(self):
    return abs(self.population[-1].fitness - self.population[0].fitness) 

  def step(self, num_steps=1, fitness='sum', selection='greatest', 
           crossover_point=None, trim='greatest'
           ):
    for i in range(num_steps):
      self.update_fitness(fitness)
      self.select(selection)
      self.crossover(crossover_point)
      self.mutation()
      self.trim(trim)
      self.update_fitness(fitness)
      
  def __str__(self):
    pop = []
    for i in range(len(self.population)):
      pop.append(str(i) + ': ' + str(self.population[i])
      + ' fit: ' + str(self.population[i].fitness))
    return str(pop)

In [ ]:
G = GeneticAlgorithm()

# loop while difference in fitness between most and least fit individuals 
# is more than 1
G.step()
while G.fitness_difference() >= 1:
  G.step()
  print(G)


['0: [1 0 0 0 0 1 1 0 0 1] fit: 4', '1: [1 0 0 1 0 1 0 1 0 0] fit: 4', '2: [1 0 1 1 0 0 1 0 1 0] fit: 5', '3: [1 1 0 0 1 1 1 1 0 0] fit: 6', '4: [0 1 1 1 1 0 0 1 0 1] fit: 6', '5: [0 1 1 1 1 1 1 1 1 1] fit: 9', '6: [0 1 1 1 1 1 1 1 1 1] fit: 9', '7: [0 1 1 1 1 1 1 1 1 1] fit: 9', '8: [0 1 1 1 1 1 1 1 1 1] fit: 9', '9: [0 1 1 1 1 1 1 1 1 1] fit: 9']
['0: [1 0 1 1 0 0 1 0 1 0] fit: 5', '1: [1 1 0 0 1 1 1 1 0 0] fit: 6', '2: [0 1 1 1 1 0 0 1 0 1] fit: 6', '3: [0 1 1 1 1 1 1 1 1 1] fit: 9', '4: [0 1 1 1 1 1 1 1 1 1] fit: 9', '5: [0 1 1 1 1 1 1 1 1 1] fit: 9', '6: [0 1 1 1 1 1 1 1 1 1] fit: 9', '7: [0 1 1 1 1 1 1 1 1 1] fit: 9', '8: [1 0 0 0 0 0 1 1 0 0] fit: 3', '9: [1 0 0 0 0 0 0 0 1 0] fit: 2']
['0: [1 0 1 1 0 0 1 0 1 0] fit: 5', '1: [1 1 0 0 1 1 1 1 0 0] fit: 6', '2: [0 1 1 1 1 0 0 1 0 1] fit: 6', '3: [0 1 1 1 1 1 1 1 1 1] fit: 9', '4: [0 1 1 1 1 1 1 1 1 1] fit: 9', '5: [0 1 1 1 1 1 1 1 1 1] fit: 9', '6: [0 1 1 1 1 1 1 1 1 1] fit: 9', '7: [0 1 1 1 1 1 1 1 1 1] fit: 9', '8: [0 1 1 1 1 1 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


In [ ]:
# testing

G = GeneticAlgorithm()
print(G)

G.update_fitness()
print(G)

G.select()
print(G)

G.crossover()
print(G)  

G.mutation()
print(G)

G.trim()
print(G)

['0: [0 1 1 0 1 1 1 1 1 1] fit: None', '1: [1 0 0 1 0 0 0 0 0 1] fit: None', '2: [0 1 1 0 0 1 1 1 1 0] fit: None', '3: [1 0 1 0 1 1 0 1 1 0] fit: None', '4: [0 1 0 1 1 1 1 1 0 1] fit: None', '5: [0 1 1 1 1 0 1 0 0 1] fit: None', '6: [1 0 1 0 1 0 0 0 0 0] fit: None', '7: [1 1 0 0 0 1 1 0 1 0] fit: None', '8: [0 1 0 1 1 1 1 1 1 0] fit: None', '9: [1 1 0 0 1 0 0 1 1 0] fit: None']
['0: [0 1 1 0 1 1 1 1 1 1] fit: 8', '1: [1 0 0 1 0 0 0 0 0 1] fit: 3', '2: [0 1 1 0 0 1 1 1 1 0] fit: 6', '3: [1 0 1 0 1 1 0 1 1 0] fit: 6', '4: [0 1 0 1 1 1 1 1 0 1] fit: 7', '5: [0 1 1 1 1 0 1 0 0 1] fit: 6', '6: [1 0 1 0 1 0 0 0 0 0] fit: 3', '7: [1 1 0 0 0 1 1 0 1 0] fit: 5', '8: [0 1 0 1 1 1 1 1 1 0] fit: 7', '9: [1 1 0 0 1 0 0 1 1 0] fit: 5']
['0: [1 0 0 1 0 0 0 0 0 1] fit: 3', '1: [1 0 1 0 1 0 0 0 0 0] fit: 3', '2: [1 1 0 0 0 1 1 0 1 0] fit: 5', '3: [1 1 0 0 1 0 0 1 1 0] fit: 5', '4: [0 1 1 0 0 1 1 1 1 0] fit: 6', '5: [1 0 1 0 1 1 0 1 1 0] fit: 6', '6: [0 1 1 1 1 0 1 0 0 1] fit: 6', '7: [0 1 0 1 1 1 1 1 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if __name__ == '__main__':
